### Data Filling

This notebook is going to be used to fill out those rows that have 0 values for all count of orcas, Adults, Juveniles and those of uncertain age.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('../data/clean_data.csv')
df.head()

,date,Latitude,Longitude,Followed_GTOA_Protocol,Interaction_time,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour
0,2023-11-01 22:15:00,32.791633,-9.906633,No,0-10,Sail,10-12.5,No,No,No,1,0,0,Spade,Sailing,5 - 7,Moderate,5 - 6 (17 - 27 knots),Night,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,"Orca interaction at 10:15pm on 01/11, 40 miles...",I would describe the behaviour of the Orca dur...
1,2023-10-31 07:50:00,39.433333,-9.383333,Yes,0-10,Sail,12.5-15,No,No,Yes,2,5,0,Twin rudder,Motoring,5 - 7,Rough,3 - 4 (7 - 16 knots),Day,50 - 75%,2 - 5,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",No,We had sandbags on our sugar scoops and metal ...,Juveniles hitting the rudders adults close by
2,2023-09-19 11:00:00,37.666667,-8.900000,No,0-10,Sail,12.5-15,No,No,Yes,1,0,0,Spade,Motoring,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Other,"Yes, moderate - immediate repairs required",No,We saw the orca approach from 10 o’clock posit...,There was an initial approach 45 minutes earli...
3,2023-09-01 13:15:00,45.600000,-3.750000,Yes,10-30,Sail,15+,No,Yes,Yes,1,2,0,Spade,Sailing,3 - 4,Calm,3 - 4 (7 - 16 knots),Day,25 - 50%,Over 10,200m+,Off,Off,White/light,Black,"Yes, moderate - immediate repairs required",No,Les trois orques passent constamment de bâbord...,Pas de comportement visblement agressif./// No...
4,2023-09-02 03:45:00,42.750000,-9.233333,Yes,0-10,Sail,12.5-15,No,No,Yes,1,2,0,Spade,Motorsailing,5 - 7,Calm,0 - 2 (0 - 6 knots),Night,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",Yes,Arrêt du pilote automatique a la 2 eme interac...,Approche furtive à la première interaction dir...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           153 non-null    object 
 1   Latitude                       153 non-null    float64
 2   Longitude                      153 non-null    float64
 3   Followed_GTOA_Protocol         153 non-null    object 
 4   Interaction_time               153 non-null    object 
 5   Boat_Type                      153 non-null    object 
 6   Boat_Length                    153 non-null    object 
 7   Towing_Inflatable              153 non-null    object 
 8   Trailing_Fishing_Lure          153 non-null    object 
 9   Physical_Contact_With_Boat     153 non-null    object 
 10  Number_of_Adult_Orcas          153 non-null    int64  
 11  Number_of_Juvenile_Orcas       153 non-null    int64  
 12  Number_of_Uncertain_Age_Orcas  153 non-null    int

In [4]:
# Desconozco por qué me ha cambiado el tipo de dato de la columan date a tipo objeto pero voy a devolverla a datetime y prosigo con la tarea
# principal de este notebook.

df.date = df.date.astype('datetime64[ms]')
print(df.date.dtype)

datetime64[ms]


Vamos a filtrar aquellas filas que tengan un 0 en las tres columnas, y de ahí quiero mostrar únicamente las columnas con Crew_Response y Orcas_Behaviour ya que es de donde puedo sacar información de cara a rellenar las otras. Se van a tomar las siguientes hipótesis para ser coherentes en el estudio:
* Solamente en caso de no decir el patron nada al respecto de cuantas orcas había, se rellenará el valor de 'Uncertain' con un valor de 1 por haberse dado una interacción. 
* En caso de mencionar el patrón un número pero no la edad o el tamaño, se meterán en la columna de 'Uncertain'
* En caso de decir el patrón que las orcas eran de tamaño considerable/grande, se considerarán estas orcas adultas.
* En caso de decir el patrón que las orcas eran pequeñas o crías, se considerán estas orcas como juveniles.

In [5]:
# Creamos un dataframe separado únicamente para mostrar aquellas filas que tengan un 0 en las tres columnas de recuento de orcas

# Luego iremos rellenando sobre el dataframe original df.

# Filas filtradas donde todos los recuentos sean 0
filtered_rows = df[(df['Number_of_Adult_Orcas'] == 0) & 
                   (df['Number_of_Juvenile_Orcas'] == 0) & 
                   (df['Number_of_Uncertain_Age_Orcas'] == 0)]

# Hago print de todas las filas que cumplan la condición, con estas 5 columnas me vale por querer ver unicamente los comentarios
print(filtered_rows[['Crew_Response', 'Orcas_Behaviour']])


                                         Crew_Response  \
23                                             Unknown   
25   Had them very close, quite a large pod checkin...   
26   Au petit matin, alors que nous avions traversé...   
27   The orcas attacked the boat directly on the fi...   
28   Orca bumped boat 'lightly' - we weren't 100% s...   
29   3 orques sont venus autour du bateau. Le plus ...   
30   Approaching the TSS we had come from Lanzerote...   
31   We were sailing about course 210 with 2 person...   
32   Motoring astern about 15 min , stopping engine...   
33   Coming from Lagos we wanted to sail as near as...   
34   Without any warning, the orcas submerged and a...   
35   5 minutes before the 2 female Orcas approached...   
36   We saw three Orcas. One older/bigger and two y...   
37   Nous marchions à la voile à 9nds sous pilote. ...   
38   Nous avons surveillés, ma femme et moi même, l...   
39   Navigation equipment switched off, going rever...   
40   One or tw